# Mecanismo preventivo de fiscalización en infraestructura pública basado en inteligencia artificial

Uso de dispositivos de fiscalización remota (DIFIR)






# 1.)Preparación de librerías y modelo

Para efectos de este análisis se trabajará con la plataforma MXNet para CUDA (https://mxnet.apache.org/versions/1.4.1/install/index.html?platform=Linux&language=Python&processor=GPU), y el modelo yolo v3 (https://cv.gluon.ai/model_zoo/detection.html) con el fin de detectar tres etiquetas (personas, equipo pesado y vehículos)

In [ ]:
%%capture
!pip install mxnet-cu112
!pip install gluoncv

In [ ]:
%%capture

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv as gcv
import hashlib
from pylab import rcParams
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils
import numpy as np
import os
from pathlib import Path
import pandas as pd
import time
import cv2

rcParams['figure.figsize'] = 5, 10

In [ ]:
network = gcv.model_zoo.get_model('yolo3_darknet53_coco', pretrained=True,ctx=mx.gpu(0))

100%|██████████| 224190/224190 [00:04<00:00, 52822.09KB/s]


In [ ]:
network.reset_class(classes=['truck', 'person','car'], reuse_weights=['truck', 'person',"car"])

#2.)Parámetros de detección



Se brinda cierta parametrización para efectos del conteo en la matriz final, etiqueta a contabilizar y tolerancia de probabilidad mínima aceptable.

Debe escoger la etiqueta y correr la flecha de la izquierda de la celda.

In [ ]:
#@title Objetos a ubicar en la imagen
objetos = "todos" #@param ["truck", "person","car","todos"]
print('Buscar', objetos)

Buscar todos


In [ ]:

if objetos != "todos":
  objeto=[objetos]
else:
  objeto=['truck', 'person','car']


In [ ]:
#@title Probabilidad mínima de que sea el objeto
#@markdown Los modelos de reconocimiento establecen una probabilidad que el objeto
#@markdown detectado corresponda a la etiqueta, debe especificarla y correr la flecha de la celda


probabilidad= 60 #@param {type:"slider", min:30, max:90, step:1}
print("Probabilidad mínima",probabilidad)

Probabilidad mínima 60


Se monta el google drive, debe poner las imágenes disponibles en https://github.com/alejoherrera/DIFIR_publico/tree/main/frames_iniciales en la carpeta de Google Drive que desea trabajar.

# Preparación de infraestructura y datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Puede definir una nueva carpeta para las imágenes procesadas, sino por defecto se guardarán en "content/frames_procesadas/". Recordar que esa carpeta es temporal

In [ ]:
salida="/content/drive/MyDrive/GIZ/Resultados/Frames_procesadas/"

In [ ]:
from os import walk
path="/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales"
imgs =next(walk(path), (None, None, []))[2]

Se verifica que se accesaron las imágenes

In [ ]:
imgs[0]

'3095.jpg'

Se establecen las funciones para lectura y transformación de las imágenes

In [ ]:
def load_image(filepath):
    
    im=image.imread(filepath)
    return im
    

In [ ]:
def transform_image(array):
    norm_image,image=data.transforms.presets.yolo.transform_test(array)
    norm_image= norm_image.as_in_context(mx.gpu(0))
    return norm_image,image

Se definen las funciones de detección y conteo de objetos

In [ ]:
def detect(network, data):
    pred=network(data)
    class_ids,scores,bounding_boxes=pred
    return class_ids, scores, bounding_boxes

In [ ]:
def count_object(network, class_ids, scores, bounding_boxes, object_label, threshold):
    idx=0
    for i in range(len(network.classes)):
        if network.classes[i]==object_label:
            idx=i
    scores=scores[0]
    class_ids=class_ids[0]
    num_object=0
    for i in range(len(scores)):
        proba=scores[i].astype('float32').asscalar()
        if proba>threshold and class_ids[i].asscalar()==idx:
            num_object+=1
    return num_object

Generación de la matriz de trazabilidad

In [ ]:
difir = Path(path)

In [ ]:
Difir1=[]
Difir1=pd.DataFrame(Difir1,columns=("camara", "latitud", "longitud","proyecto","etiqueta","probabilidad","cantidad_detectada","anio","mes","dia","hora","minuto","archivo_entrada","archivo_salida"))

In [ ]:
threshold=probabilidad/100#0.6

for filepath in difir.glob('**/*.jpg'):

  print(filepath)
  im=load_image(filepath)
  norm_image, unnorm_image=transform_image(im)
  file=str(filepath)
  class_ids, scores, bounding_boxes =detect(network,norm_image)

 

  #for object_label in ["truck","person","car"]:
  for object_label in objeto:
 
    count = count_object(network, class_ids, scores, bounding_boxes, object_label,threshold)
    #ax = utils.viz.plot_bbox(unnorm_image, bounding_boxes[0], scores[0], class_ids[0], class_names=network.classes)
    #fig = plt.gcf()
    archivo=file.split("/",7)
    archivo=archivo[-1]
    procesada=salida+archivo
    #plt.savefig(procesada)
    #time.sleep(1)
    

 

    Difir1.loc[len(Difir1)] = ["DIFIR1",9.9322,-84.10,"Prueba_GIZ",object_label,threshold,count,file[67:71],file[72:74],file[75:77],file[78:80],file[81:83],file,procesada]
    Difir1.to_excel(salida+"DIFIR.xlsx")
    time.sleep(1)
    #print("{} objects of class '{}' detected".format(count, object_label))

/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3095.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3096.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3097.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3098.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3099.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3100.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3101.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3102.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3103.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3104.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3105.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3106.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3107.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3108.jpg
/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales/3109.jpg
/content/drive/MyDrive/GIZ/Resultados/Fr

In [ ]:
Difir1

In [ ]:
Difir1.to_excel(salida+"DIFIR.xlsx")